In [1]:
import sys
sys.path.append('..')

In [1]:
from spinesUtils import *
import pandas as pd

In [4]:
%timeit -r 1 -n 1 dataloader('/Users/guobingming/Downloads/dataset.csv', turbo_method='polars', transform2low_mem=True, verbose=True)
%timeit -r 1 -n 1 dataloader('/Users/guobingming/Downloads/dataset.csv', turbo_method='pyarrow', transform2low_mem=True, verbose=True)

Converting ...:   0%|          | 0/193 [00:00<?, ?it/s]

Memory usage before conversion is: 2416.73 MB  
Memory usage after conversion is: 1344.8 MB  
After conversion, the percentage of memory fluctuation is 44.4 %


16 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Converting ...:   0%|          | 0/193 [00:00<?, ?it/s]

4.25 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Memory usage before conversion is: 2416.73 MB  
Memory usage after conversion is: 1344.8 MB  
After conversion, the percentage of memory fluctuation is 44.4 %


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 205 entries, dt to next_login_7ds_freq
dtypes: float16(178), float32(14), float64(1), object(12)
memory usage: 492.1+ MB


In [19]:
y_col = 'is_losing_user'

In [20]:
x_cols = get_x_cols(df[select_numeric_cols(df)], y_col, exclude_cols=['dt', 'userid'])

In [21]:
classify_samples_dist(df, y_col)

,sample_count,rate
is_losing_user,,
0.0,953048,95.3%
1.0,46952,4.7%


In [22]:
from lightgbm import LGBMClassifier
classifier = BinaryBalanceClassifier(meta_estimators=[LGBMClassifier(), LGBMClassifier()], verbose=2)

In [23]:
classifier.fit(df[x_cols], df[y_col], threshold_search_set=(df[x_cols], df[y_col]))

17:33:06 2023-09-20 - [Main step] X type is pandas dataframe, y type is pandas series.
17:33:06 2023-09-20 - [Category columns] 
17:33:06 2023-09-20 - [Main step] Prepare to fit.
17:33:06 2023-09-20 - [Model L1] Training samples shape is (1000000, 192), sample size ratio is {0.0: 0.95, 1.0: 0.05}
17:33:06 2023-09-20 - [Main step] Model L1 fitted.
17:33:06 2023-09-20 - [Main step] Datasets split for model L2.
17:33:07 2023-09-20 - [Model L2] Training samples shape is (93904, 192), sample size ratio is {0.0: 0.5, 1.0: 0.5}
17:33:09 2023-09-20 - [Main step] Model L2 fitted.
17:33:11 2023-09-20 - [Main step] Datasets split for model L3.
17:33:11 2023-09-20 - [Model L3] Training samples shape is (273727, 192), sample size ratio is {0.0: 0.86, 1.0: 0.14}
17:33:15 2023-09-20 - [Main step] Model L3 fitted.
17:33:15 2023-09-20 - [Main step] start to search probability threshold...
17:33:15 2023-09-20 - Automatically searching...
17:33:17 2023-09-20 - [current loop] try threshold: 0.0, max f1 sc

In [24]:
classifier.auto_threshold

frozendict.frozendict({'Model L2': 0.8, 'Model L3': 0.17439999999999997})

In [25]:
from sklearn.metrics import f1_score, recall_score, precision_score
(
    'f1:', f1_score(df[y_col], classifier.predict(df[x_cols])), 
    'recall:', recall_score(df[y_col], classifier.predict(df[x_cols])), 
    'precision:', precision_score(df[y_col], classifier.predict(df[x_cols]))
)

17:34:08 2023-09-20 - [Main step] Prepare to predict. Predict mode:soft
17:34:08 2023-09-20 - [Main step] Model L1 has been predicted.
17:34:10 2023-09-20 - [Main step] Model L2 has been predicted.
17:34:12 2023-09-20 - [Main step] Model L3 has been predicted.
17:34:12 2023-09-20 - [Main step] Prepare to predict. Predict mode:soft
17:34:12 2023-09-20 - [Main step] Model L1 has been predicted.
17:34:14 2023-09-20 - [Main step] Model L2 has been predicted.
17:34:16 2023-09-20 - [Main step] Model L3 has been predicted.
17:34:17 2023-09-20 - [Main step] Prepare to predict. Predict mode:soft
17:34:17 2023-09-20 - [Main step] Model L1 has been predicted.
17:34:18 2023-09-20 - [Main step] Model L2 has been predicted.
17:34:20 2023-09-20 - [Main step] Model L3 has been predicted.


('f1:',
 0.3490528012898025,
 'recall:',
 0.3688873743397512,
 'precision:',
 0.3312423500611995)

In [26]:
from sklearn.metrics import f1_score, recall_score, precision_score
lgb = LGBMClassifier()
lgb.fit(df[x_cols], df[y_col])
(
    'f1:', f1_score(df[y_col], lgb.predict(df[x_cols])), 
    'recall:', recall_score(df[y_col], lgb.predict(df[x_cols])), 
    'precision:', precision_score(df[y_col], lgb.predict(df[x_cols]))
)

('f1:',
 0.08188590511194103,
 'recall:',
 0.04413017549838132,
 'precision:',
 0.5668946648426813)